In [38]:
from keras.layers import *
from keras.models import *
from keras.backend import sum as Sum
from keras.backend import ones


In [4]:
featurelen=2048
dictlen=8495
sentencelen=39
EMBEDDING_DIM=512
visibletime=1

cnn_X = 12
cnn_Y = 12
filters = 768



In [ ]:
def get_model():
    word_inp = Input((sentencelen,))
    emd_word = Embedding(dictlen, 256)(word_inp)
    emd_word = BatchNormalization()(emd_word)
    
    img_inp = Input((filters,cnn_X*cnn_Y))
    img_inp = RepeatVector(sentencelen)(img_inp)

    
    att_inp = Input((sentencelen, cnn_X*cnn_Y)) # Probabilities
    
    att_inp = TimeDistributed(RepeatVector(filters))(att_inp)
    
    z_vec = TimeDistributed(Multiply())([img_inp, att_inp])
    
    z_t = TimeDistributed(sum(keep_dims=False))[z_vec]

    
    
    
    return model

In [40]:
word_inp = Input((sentencelen,))
emd_word = Embedding(dictlen, 256)(word_inp)
emd_word = BatchNormalization()(emd_word)

img_in = Input((filters,cnn_X*cnn_Y))
img_inp = Flatten()(img_in)
img_inp = RepeatVector(sentencelen)(img_inp)
img_inp = Reshape((sentencelen,filters,cnn_X*cnn_Y))(img_inp)


att_in = Input((sentencelen, cnn_X*cnn_Y)) # Probabilities
att_inp = Flatten()(att_in)
att_inp = RepeatVector(filters)(att_inp)
att_inp = Reshape((sentencelen,filters,cnn_X*cnn_Y))(att_inp)

mult  = multiply([img_inp, att_inp])
z = Sum(mult, axis=3)

lstm_in = concatenate([z, emd_word])
lstm_out = LSTM(1536,return_sequences=True,dropout=0.5)(lstm_in)

model = Model(inputs=[word_inp, img_in, att_in],outputs=lstm_out)

IMG_INP = Input((filters,cnn_X*cnn_Y))
WORD_INP = Input((sentencelen,))

ini_ones = ones((cnn_X*cnn_Y,))

LSTM_OUT = model([WORD_INP, IMG_INP, ini_ones ])

attend = TimeDistributed(Dense(cnn_X*cnn_Y,activation="softmax"))(LSTM_OUT)

LSTM_OUT = model([WORD_INP, IMG_INP, attend])

fin = TimeDistributed(Dense(dictlen, activation="softmax"))(LSTM_OUT) 

fin_model = Model(inputs=[IMG_INP, WORD_INP], outputs=fin)


AttributeError: 'Tensor' object has no attribute '_keras_history'